Making the

In [7]:
import re
import pyrfume
import numpy as np
import pandas as pd
import seaborn as sns
import rdkit.Chem as Chem
import matplotlib.pyplot as plt
from rdkit.Chem import rdMolDescriptors
from sklearn.preprocessing import MultiLabelBinarizer
from computer_ontology.config import lemma_path, expert_path
from computer_ontology.custom_funcs import get_unique

In [65]:
from os import replace
def leffingwell_reverse_one_hot(row):
    """
    Takes a row of the Leffingwell dataset
    and reverses one-hot-encoding.

    :param row: A given row of the Leffingwell dataset.
    :type row: pandas Dataframe row
    :return: A list of classes/labels for each row.
    :rtype: List
    """
    labels = [col for col in leffingwell.columns if row[col] == 1]
    return ';'.join(labels)

def dravnieks_top_n_columns(row, n):
    """
    This is for the Dravnieks dataset
    since its format is unique. It
    selects the labels with the highest
    values and returns them in the standard
    format used for the other datasets.

    :param row: A given row of the Dravnieks dataset.
    :type row: pandas Dataframe row
    :param n: The number of labels to consider.
    :type n: int
    :return: top n labels.
    :rtype: string
    """
    sorted_columns = row.sort_values(ascending=False).index
    top_n = sorted_columns[:n]
    return ';'.join(top_n)

def get_unique(df):
    """
    This function takes in an odor dataset as a
    dataframe and returns a dataframe containing
    all the unique labels of the input dataframe.

    :requirements: labels should be called 'Descriptors'.

    :param df: A multilabel dataframe with labels separated by ';'
    :type df: pandas Dataframe
    :return: A list containing the unique descriptors.
    :rtype: list
    """
    all_descriptors = []

    for des in df['Descriptors']:
        all_descriptors.extend(des.split(';'))

    unique_descriptors = list(set(all_descriptors))
    unique_descriptors.sort()

    return unique_descriptors

def get_dataset(name):
    """
    This function takes in a string which is the
    name of the dataset and returns the fetched
    dataset.

    :param name: Name of the dataset according to Pyrfume
    :type name: string
    :return: The dataset called
    :rtype: Dataframe
    """
    # Load molecular and stimulus data
    mols =  pyrfume.load_data(f'{name}/molecules.csv')["IsomericSMILES"]
    stim =  pyrfume.load_data(f'{name}/stimuli.csv')

    # Deal with exceptions for behavior data
    try:
      behav =  pyrfume.load_data(f'{name}/behavior.csv')
    except:
      try:
        behav =  pyrfume.load_data(f'{name}/behavior_1_sparse.csv')
      except:
        behav =  pyrfume.load_data(f'{name}/behavior_1.csv')

    if name == 'ifra_2019':
      behav['Descriptor 1'] = behav[['Descriptor 1', 'Descriptor 2', 'Descriptor 3']].astype(str).apply(';'.join, axis=1)

      behav = behav['Descriptor 1']

    labels = pd.merge(stim, behav, on='Stimulus')

    # Deal with exceptions during Merging
    try:
      df = pd.merge(mols, labels, on='CID')
    except:
      labels.rename(columns={'new_CID': 'CID'}, inplace=True)
      df = pd.merge(mols, labels, on='CID')

    return df

def check_and_replace(description):
    """
    Iterates through a given ";" separated strings
    and replaces them with the mapping assigned by
    any list labelled "mapping".

    :param description: Text separated by ';'
    :type name: string
    :return: Text replaced according to the mapping
    :rtype: string
    """
    descriptors = description.split(';')
    new_descriptors = []

    for descriptor in descriptors:
        for cluster in clusters:
            if descriptor in cluster:
                new_descriptors.append(cluster[0])

    return ';'.join(new_descriptors)

def make_unique(labels):
   """
   Takes a text separated by ";" and makes them
   unique.

   :param description: Text separated by ';'.
   :type name: string
   :return: words within text made unique.
   :rtype: string
    """
   return ';'.join(list(set(labels.split(';'))))

def count_words(label_str):
    """
    Takes a ";" separated strings and counts them

   :param description: Text separated by ';'.
   :type name: string
   :return: number of words separated by ";"
   :rtype: int
    """
    return len(label_str.split(';'))

In [66]:
# Arctander
arctander = get_dataset('arctander_1960')
arctander.drop(['ChemicalName', 'CAS'], axis=1, inplace=True)
arctander.rename(columns={'Labels': 'Descriptors'}, inplace=True)
arctander = arctander.dropna()

# AromaDB
aromadb = get_dataset('aromadb')
aromadb.drop(['Raw Descriptors', 'Modifiers'], axis=1, inplace=True)
aromadb.rename(columns={'Filtered Descriptors': 'Descriptors'}, inplace=True)

# Dravnieks
dravnieks = get_dataset('dravnieks_1985')
dravnieks = dravnieks.drop(["Name", "Conc", "CAS"], axis=1)
dravnieks['Descriptors'] = dravnieks.drop(["CID", "IsomericSMILES"], axis=1).apply(lambda row: dravnieks_top_n_columns(row, 3), axis=1)
dravnieks = dravnieks[['CID', 'IsomericSMILES', 'Descriptors']]

# FlavorDB
flavordb = get_dataset('flavordb')
flavordb = flavordb[['CID', 'Odor Percepts', 'IsomericSMILES']]
flavordb.rename(columns={'Odor Percepts': 'Descriptors'}, inplace=True)
flavordb.rename(columns={'IsomericSMILES': 'IsomericSMILES'}, inplace=True)
flavordb = flavordb.dropna()

# Flavornet
flavornet = get_dataset('flavornet')
flavornet.rename(columns={'Descriptors':'Descriptors'}, inplace=True)
flavornet.rename(columns={'IsomericSMILES': 'IsomericSMILES'}, inplace=True)

# Goodscents
goodscents = get_dataset('goodscents')
goodscents.drop(['TGSC ID', 'Concentration %','Solvent'], axis=1, inplace=True)
goodscents = goodscents.dropna()

# IFRA
ifra = get_dataset('ifra_2019')
ifra.rename(columns={'Descriptor 1': 'Descriptors'}, inplace=True)

# Leffingwell
leffingwell = get_dataset('leffingwell')

#This filtering was done as CIDs below 0 were not well documented.
leffingwell = leffingwell[leffingwell['CID']>0]

# Apply the function to create the 'labels' column
leffingwell['Descriptors'] = leffingwell.apply(leffingwell_reverse_one_hot, axis=1)

leffingwell.rename(columns={'IsomericSMILES_x':'IsomericSMILES'}, inplace=True)
leffingwell = leffingwell[['CID', 'IsomericSMILES', 'Descriptors']]
leffingwell = leffingwell.dropna()


Merging the Datasets.

In [67]:
# Concatentating all datasets
all_compounds = pd.concat([arctander, aromadb, dravnieks, flavordb, flavornet, goodscents, ifra, leffingwell], axis=0, ignore_index=True)

# Here all rows are merged on CID and labels are joined together.
all_compounds = all_compounds.astype({'CID': 'int64'}) #Setting the CID column to dtype int64 because it works better with other code.

all_compounds.reset_index(drop=True, inplace=True)
agg_functions = {'CID': 'first', 'IsomericSMILES': 'first', 'Descriptors': lambda x: ';'.join(x)} #Takes the first CID and SMILES and joins all the descriptors together.
all_compounds = all_compounds.groupby('CID').aggregate(agg_functions)
print(all_compounds.shape)

(7541, 3)


Removing SMILES with dots in them as they are mixtures.

In [68]:
#Get SMILES with '.' in them
all_compounds['HasDot'] = all_compounds['IsomericSMILES'].apply(lambda x: '.' in x)

#Drop all rows with '.' in SMILES
all_compounds = all_compounds[all_compounds['HasDot'] == False]
all_compounds.drop(['HasDot'], axis=1, inplace=True)
print(all_compounds.shape)

(7267, 3)


Removing Capitalization and Duplicates in the labels.

In [69]:
all_compounds['Descriptors'] = all_compounds['Descriptors'].str.lower()
all_compounds['Descriptors'] = all_compounds['Descriptors'].str.split(';').apply(set).apply(list).apply(lambda x: ';'.join(x))

Lemmatization step

In [2]:
#Reading in the labels to be lemmatized.
f = open(lemma_path, "r")

labels_to_be_lemmatized = []
for line in f:
    labels_to_be_lemmatized.append((line.strip()))

f.close()

for label in labels_to_be_lemmatized:
    temp = label.split(',')
    all_compounds['Descriptors'] = all_compounds['Descriptors'].apply(lambda x: x.replace(temp[0], temp[1]))

NameError: name 'all_compounds' is not defined

In [71]:
all_compounds['Descriptors'] = all_compounds['Descriptors'].apply(lambda x: x.lower())
all_compounds['Descriptors'] = all_compounds['Descriptors'].apply(make_unique)
all_compounds['Descriptors'] = all_compounds['Descriptors'].dropna()
all_compounds = all_compounds[all_compounds['Descriptors'] != '']
print(all_compounds.shape)

(7267, 3)


In [72]:
all_compounds['Descriptors'] = all_compounds['Descriptors'].apply(lambda x: x.replace('odorless;', ''))
all_compounds['Descriptors'] = all_compounds['Descriptors'].apply(lambda x: x.replace(';odorless', ''))
all_compounds['Descriptors'] = all_compounds['Descriptors'].apply(lambda x: x.replace('odorless', ''))
print(all_compounds.shape)

(7267, 3)


Remove odor descriptors that occur less than 30 times

In [73]:
#Checks descriptor frequency
labels_to_remove = all_compounds['Descriptors'].str.split(';', expand=True).stack().value_counts()

#Remove labels that occur less than 30 times
labels_to_remove = labels_to_remove[labels_to_remove < 30]
all_compounds['Descriptors'] = all_compounds['Descriptors'].apply(lambda x: ';'.join([item for item in x.split(';') if item not in labels_to_remove.index]))
all_compounds = all_compounds[all_compounds['Descriptors'] != '']
print(all_compounds.shape)

(7038, 3)


Remove compounds that are heavier than 296 g/mol as it is the heaviest odorant.

In [74]:
#Get molecular weights of compounds.
all_compounds['MolecularWeight'] = all_compounds['IsomericSMILES'].apply(lambda x: rdMolDescriptors.CalcExactMolWt(Chem.MolFromSmiles(x)))

#Remove compounds with molecular weights above 296.
all_compounds = all_compounds[all_compounds['MolecularWeight'] <= 296]
all_compounds = all_compounds.drop(['MolecularWeight'], axis=1)
print(all_compounds.shape)

(6834, 3)


In [75]:
#Remove compounds containing other elements besides C, N, O, S, P.
all_compounds['HasOtherElements'] = all_compounds['IsomericSMILES'].apply(lambda x: Chem.MolFromSmiles(x).HasSubstructMatch(Chem.MolFromSmarts('[!#6;!#7;!#8;!#16;!#15]')))

all_compounds = all_compounds[all_compounds['HasOtherElements'] == False]
all_compounds.drop(['HasOtherElements'], axis=1, inplace=True)
print(all_compounds.shape)

(6808, 3)


<ipython-input-75-1049f10d3370>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_compounds.drop(['HasOtherElements'], axis=1, inplace=True)


One hot encode the labels

In [76]:
combined_dataset = all_compounds.copy()

#Turn descriptors column into lists
combined_dataset['Descriptors'] = combined_dataset['Descriptors'].apply(lambda x: x.split(';'))

mlb = MultiLabelBinarizer(sparse_output=True)
mlb.fit(combined_dataset['Descriptors'])

data_bin = combined_dataset.join(pd.DataFrame.sparse.from_spmatrix(mlb.transform(combined_dataset['Descriptors']), index=combined_dataset.index, columns=mlb.classes_))

# uncomment to download
# data_bin.to_csv('combined_dataset.csv', index=False)

## Getting the dataset with a Expert derived Ontology (CO)

In [8]:
f = open(expert_path, 'r')
lines = f.readlines()

clusters = []

for line in lines:
    line = line.lower()
    line = line.strip()
    line = line.split(',')
    clusters.append(line)

labels_in_data = get_unique(all_compounds)
temp = []

for cluster in clusters:
    temp.extend(cluster)

diff = list(set(labels_in_data) - set(temp))

NameError: name 'all_compounds' is not defined

In [78]:
# Replacing the odor descriptors with the umbrella terms
umbrella_dataset = all_compounds.copy()

umbrella_dataset['Descriptors'] = umbrella_dataset['Descriptors'].apply(check_and_replace)
umbrella_dataset['Descriptors'] = umbrella_dataset['Descriptors'].apply(make_unique)
umbrella_dataset['Descriptors'] = umbrella_dataset['Descriptors'].dropna()
umbrella_dataset = umbrella_dataset[umbrella_dataset['Descriptors'] != '']
print(umbrella_dataset.shape)

(6435, 3)


In [82]:
umbrella_dataset

,CID,IsomericSMILES,Descriptors
CID,,,
49,49,CC(C)C(=O)C(=O)O,fruity
58,58,CCC(=O)C(=O)O,savory
70,70,CC(C)CC(=O)C(=O)O,fruity
98,98,C(C(=O)C(=O)O)S,savory
107,107,C1=CC=C(C=C1)CCC(=O)O,flower;savory;animal;spices
...,...,...,...
159578871,159578871,CCC(CO)C1CCC(CC1)C,flower
161504856,161504856,CCCCCCCC=COC(=O)CCC,green;fruity
162353069,162353069,C(CSCCC(S)S)C(S)S,green;fruity;savory
